NECESARIO PARA EJECUTAR OPENCL

In [2]:



!pip install pyopencl


LIBRERIAS PARA IMPORTAR

In [4]:
import pyopencl as cl
import numpy as np
import pandas as pd
from PIL import Image, ImageFilter

C:\Users\Eevee\anaconda3\Lib\site-packages\pytools\persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


KERNEL FILTROS BLANCO Y NREGO

In [6]:
kernel_filter_black_white="""
__kernel void kernel_filter_black_white(__global uchar* imagen_in,__global uchar* imagen_out,__constant float* filtro,int dim,int ancho,int alto){

  int fila = get_global_id(0);
  int columna = get_global_id(1);

  int centro=(dim-1)/2;
  float suma=0.0f;
  int i,j;

  //Asegurarse de que el píxel esté dentro de los límites

   if (centro <= fila && fila < (alto - centro) && centro <= columna && columna < (ancho - centro)) {

       for(i=-centro;i<=centro;i++){

            for(j=-centro;j<=centro;j++){

               float pixel_value = imagen_in[(fila+i) * ancho + (columna+j)];

                suma += pixel_value * filtro[(i + centro) * dim + (j + centro)];
                }
                }

                imagen_out[fila * ancho + columna]=(uchar)suma;
    }

  else{
    imagen_out[fila * ancho + columna]=imagen_in[fila * ancho + columna];
  }

}

"""

_

'{"dataframes": [], "user": "Eevee"}'

IMPORTAR IMAGENES

In [8]:
def importar_imagen(path):
  imagen = Image.open(path)
  imagen = imagen.convert("L")
  return imagen

FUNCION PARA APLICAR FILTRO BLANCO Y NEGRO(HAY QUE DARLE LOS FILTROS)

In [12]:
def aplicar_filtro(image_path,filtro,kernel_code,kernel_name,device_type,local_size):

  #Importar imagen
  imagen=importar_imagen(image_path)

  #Convertirla a un array
  imagen_np = np.array(imagen).astype(np.uint8)

  #Dimensiones matriz
  tam_x=imagen_np.shape[0]
  tam_y=imagen_np.shape[1]

  #Crear array imagen final
  imagen_out_np = np.empty_like(imagen_np)


  # Plataforma
  platform = cl.get_platforms()[0]

  # Dispositivo (GPU)
  device = platform.get_devices(device_type=device_type)[0]

  # Crear contexto con el dispositivo seleccionado
  context = cl.Context([device])

  # Crear una cola de comandos
  command_queue = cl.CommandQueue(context, device=device, properties=cl.command_queue_properties.PROFILING_ENABLE)

  # Crear el programa y compilarlo
  program = cl.Program(context, kernel_code)
  try:
       program.build()
  except Exception as e:
    print("Build log:")
    print(program.get_build_info(device, cl.program_build_info.LOG))
    raise e

  # Crear el kernel
  kernel_filter = cl.Kernel(program, kernel_name)


  # Definir el formato de la imagen (usaremos canales escala de grises en este ejemplo)

  image_format = cl.ImageFormat(cl.channel_order.LUMINANCE, cl.channel_type.UNSIGNED_INT8) #Changed channel_order to LUMINANCE for grayscale

  #Crear buffers imagen
  #image_in = cl.Image(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, image_format, (tam_y, tam_x), None, imagen_np)
  #image_out = cl.Image(context, cl.mem_flags.WRITE_ONLY, image_format, (tam_y, tam_x))

  # Crear buffers de imagen
  image_in = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=imagen_np)
  image_out = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, imagen_out_np.nbytes)

    # Crear buffer para el filtro


  # Create buffer for filter
  filtro_buf = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=filtro)

  # Argumentos del kernel

  kernel_filter.set_arg(0, image_in)
  kernel_filter.set_arg(1, image_out)
  kernel_filter.set_arg(2, filtro_buf)
  kernel_filter.set_arg(3, np.int32(filtro.shape[0]))
  kernel_filter.set_arg(4, np.int32(imagen_np.shape[1]))
  kernel_filter.set_arg(5, np.int32(imagen_np.shape[0]))

  # Ejecutar el kernel y registrar el evento
  global_size = (tam_x, tam_y)  # Tamaño global
  event = cl.enqueue_nd_range_kernel(command_queue, kernel_filter, global_size, local_size)

  # Esperar a que se complete el evento
  event.wait()

  # Obtener el tiempo de ejecución en nanosegundos
  exec_time = 1e-9 * (event.profile.end - event.profile.start)

  #Leer imagen
  cl.enqueue_copy(command_queue, imagen_out_np, image_out)



  #Cargar la imagen

  imagen_resultante = Image.fromarray(imagen_out_np)
  imagen_resultante.show()
  imagen_resultante.save("C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/pirineos_filtro_byn.png")


  return imagen_resultante,exec_time







EJEMPLOS FILTROS

In [14]:
filtro_mean=np.array([
    [1/9, 1/9, 1/9],
    [1/9, 1/9, 1/9],
    [1/9, 1/9, 1/9]
], dtype=np.float32)

filtro_gaussiano=np.array([
    [1/16, 2/16, 1/16],
    [2/16, 4/16, 2/16],
    [1/16, 2/16, 1/16]
], dtype=np.float32)


EJEMPLO MEAN FILTER

In [19]:
PYOPENCL_COMPILER_OUTPUT=1
device_type = cl.device_type.GPU
local_size=(16,16)
image_path="C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/pirineos_imagen.png"
imagen_resultante,exec_time=aplicar_filtro(image_path,filtro_mean,kernel_filter_black_white,"kernel_filter_black_white",device_type,local_size)

print(exec_time)


4.8032000000000004e-05


EJEMPLO GAUSSIAN FILTER

In [23]:
PYOPENCL_COMPILER_OUTPUT=1
device_type = cl.device_type.GPU
local_size=(16,16)
image_path="C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/pirineos_imagen.png"
imagen_resultante,exec_time=aplicar_filtro(image_path,filtro_gaussiano,kernel_filter_black_white,"kernel_filter_black_white",device_type,local_size)

print(exec_time)

4.7936e-05
